In [1]:
#%pip install langchain gradio chromadb pdfminer.six langchain-community notebook langchain_ollama langchain_chroma pymupdf openai langsmith langchain[openai] langchain-community openevals

In [2]:
import os
import gradio as gr
import pandas as pd
from dotenv import load_dotenv
from langchain_chroma import Chroma
from typing_extensions import Annotated, TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 2. Evaluation
from langsmith import Client, wrappers, traceable
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT

/home/adam/Documents/adam/pdfMind/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load environment variables
load_dotenv()
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
# 1. Configuration
DATA_PATH = "data"
LLM_MODEL_NAME = "mistral-nemo:latest"
EMBED_MODEL_NAME = "nomic-embed-text"
CHROMA_PATH = f"/home/adam/Documents/adam/pdfMind/chroma_db/{EMBED_MODEL_NAME}"

In [5]:
print("Creating embeddings and vector store (this may take a moment)...")
embeddings = OllamaEmbeddings(model=EMBED_MODEL_NAME)

Creating embeddings and vector store (this may take a moment)...


In [6]:
# 2. Load Documents
loader = DirectoryLoader(DATA_PATH, glob="*.pdf", loader_cls=PyMuPDFLoader)
documents = loader.load()

In [7]:
# 2. Fonction de nettoyage optimisée pour le Français
import re
def clean_french_text(text):
    text = text.replace('\xa0', ' ').replace('\n', ' ')
    text = text.replace("’", "'").replace("‘", "'")
    text = re.sub(r'(\w)-\s+(\w)', r'\1\2', text)
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text
cleaned_documents = []
for doc in documents:
    doc.page_content = clean_french_text(doc.page_content)
    if len(doc.page_content) > 20:
        cleaned_documents.append(doc)

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", ". ", " ", ""])
chunks = text_splitter.split_documents(cleaned_documents)
print(f"Split into {len(chunks)} chunks.")

Split into 305 chunks.


In [9]:
# Create Chroma vector store from documents
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=CHROMA_PATH
)

In [10]:
retriever = vector_store.as_retriever(type="similarity", search_kwargs={"k": 5})

In [11]:
llm = ChatOllama(model=LLM_MODEL_NAME)

@traceable()
def rag_bot(question: str) -> dict:
   docs = retriever.invoke(question)
   docs_string = "".join(doc.page_content for doc in docs)
   instructions = f"""Tu es un consultant expert en qualifications du bâtiment (Qualibat, RGE, Normes).
Ta mission est d'expliquer les documents techniques fournis de manière pédagogique, structurée et synthétique.

RÈGLES DE RÉDACTION (À SUIVRE IMPÉRATIVEMENT) :

1. STRUCTURE VISUELLE :
   - Commence toujours par une phrase d'introduction qui pose le contexte global.
   - Utilise des **titres de sections** clairs pour séparer les thématiques.
   - Utilise systématiquement des listes à puces (•) pour énumérer les détails.

2. PÉDAGOGIE ET DÉTAILS :
   - Si la question porte sur une **nomenclature ou un code** : Décortique la logique (ex: 1er chiffre = Famille). Donne un exemple concret (comme le code 2111 ou autre présent dans le contexte) pour illustrer.
   - Si la question porte sur des **règles/normes** : Cite précisément les références (ex: NF X 46-010) et les durées de validité.
   - Mets en **gras** les termes techniques importants, les chiffres clés et les concepts définis.

3. TON :
   - Professionnel, instructif et précis.
   - Ne dis jamais "D'après le contexte", intègre l'information comme une connaissance établie.

4. CONTRAINTE : Si la réponse n'est pas dans le contexte, dis "Je ne sais pas". N'invente rien.

CONTEXTE DOCUMENTAIRE :

{docs_string}
"""

   # langchain ChatModel will be automatically traced
   ai_msg = llm.invoke([
           {"role": "system", "content": instructions},
           {"role": "user", "content": question},
       ],
   )
   return {"answer": ai_msg.content, "documents": docs}

In [12]:
res = rag_bot("Quelles sont les 9 Famille d'activités ?")
print(res["answer"])

Les 9 familles d’activités sont :

Famille 1 | PRÉPARATION DU SITE ET INFRASTRUCTURE Famille 2 | STRUCTURE ET GROS ŒUVRE Famille 3 | ENVELOPPE EXTÉRIEURE Famille 4 | CLOS - DIVISIONS - AMÉNAGEMENTS Famille 5 | ÉNERGIES ET FLUIDES Famille 6 | FINITIONS Famille 7 | ISOLATION THERMIQUE - ACOUSTIQUE - FRIGORIFIQUE Famille 8 | PERFORMANCE ÉNERGÉTIQUE Famille 9 | AGENCEMENT ET AMÉNAGEMENT


# Evaluation

### Exactitude (Correctness): Reponse vs reponse de reference
- **Objectif** : Mesurer « la similarité/exactitude de la réponse de la chaîne RAG par rapport à une réponse de référence (vérité terrain) »
- **Mode** : Nécessite une réponse de référence (vérité terrain) fournie dans un jeu de données
- **Évaluateur** : Utiliser un LLM comme juge pour évaluer l'exactitude de la réponse.

In [13]:
# Schéma de sortie pour la notation (Correctness)
class CorrectnessGrade(TypedDict):
    explanation: Annotated[str, ..., "Expliquez votre raisonnement pour le score obtenu"]
    correct: Annotated[bool, ..., "Vrai si la réponse est correcte, Faux sinon."]

# Instructions de notation (Prompt)
correctness_instructions = """Vous êtes un enseignant qui corrige un quiz. On vous donnera une QUESTION, la RÉPONSE DE RÉFÉRENCE (correcte) et la RÉPONSE de l'ÉLÈVE. Voici les critères de notation à suivre :

(1) Notez les réponses de l'élève en vous basant UNIQUEMENT sur leur exactitude factuelle par rapport à la réponse de référence.

(2) Assurez-vous que la réponse de l'élève ne contient aucune affirmation contradictoire.

(3) Il est ACCEPTABLE que la réponse de l'élève contienne plus d'informations que la réponse de référence, tant qu'elle reste factuellement exacte par rapport à cette dernière.

Exactitude :
Une valeur d'exactitude "True" (Vrai) signifie que la réponse de l'élève répond à tous les critères.
Une valeur d'exactitude "False" (Faux) signifie que la réponse de l'élève ne répond pas à tous les critères.

Expliquez votre raisonnement étape par étape pour garantir que votre analyse et votre conclusion sont correctes. Évitez de donner simplement la réponse correcte dès le début."""

# Initialisation du LLM de notation
grader_llm = ChatOllama(model=LLM_MODEL_NAME, temperature=0).with_structured_output(
    CorrectnessGrade, method="json_schema", strict=True
)

# Évaluateur
def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """Un évaluateur pour l'exactitude de la réponse RAG par rapport à une référence."""
    # Préparation du comparatif : Question vs Référence vs Réponse générée
    answers = f"""\
QUESTION: {inputs['question']}
RÉPONSE DE RÉFÉRENCE: {reference_outputs['answer']}
RÉPONSE DE L'ÉLÈVE: {outputs['answer']}"""

    # Exécution de l'évaluateur
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions},
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

### Pertinence (Relevance): Réponse vs Entrée (Input)
Le flux est similaire à celui décrit précédemment, mais nous examinons simplement les entrées (inputs) et les sorties (outputs) sans avoir besoin des réponses de référence (reference_outputs).

Sans réponse de référence, nous ne pouvons pas évaluer l'exactitude factuelle, mais nous pouvons tout de même évaluer la pertinence — c'est-à-dire, déterminer si le modèle a répondu ou non à la question de l'utilisateur.

In [14]:
# Schéma de sortie pour la notation (Relevance)
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Expliquez votre raisonnement pour le score obtenu"]
    relevant: Annotated[
        bool, ..., "Indiquez si la réponse traite directement la question posée"
    ]

# Instructions de notation (Prompt)
relevance_instructions = """Vous êtes un enseignant qui corrige un quiz. On vous donnera une QUESTION et une RÉPONSE de l'ÉLÈVE. Voici les critères de notation à suivre :

(1) Assurez-vous que la RÉPONSE de l'ÉLÈVE est concise et pertinente par rapport à la QUESTION.
(2) Assurez-vous que la RÉPONSE de l'ÉLÈVE aide réellement à répondre à la QUESTION.

Pertinence :
Une valeur de pertinence "True" (Vrai) signifie que la réponse de l'élève répond à tous les critères.
Une valeur de pertinence "False" (Faux) signifie que la réponse de l'élève ne répond pas à tous les critères.

Expliquez votre raisonnement étape par étape pour garantir que votre analyse et votre conclusion sont correctes. Évitez de donner simplement la réponse correcte dès le début."""

# Initialisation du LLM de notation
relevance_llm = ChatOllama(model=LLM_MODEL_NAME, temperature=0).with_structured_output(
    RelevanceGrade, method="json_schema", strict=True
)

# Évaluateur
def relevance(inputs: dict, outputs: dict) -> bool:
    """Un évaluateur simple pour la pertinence et l'utilité de la réponse RAG."""
    # Préparation du contenu : Question vs Réponse générée
    answer = f"QUESTION: {inputs['question']}\nRÉPONSE DE L'ÉLÈVE: {outputs['answer']}"
    
    # Exécution de l'évaluateur
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

### Fidélité (Groundedness) : Réponse vs Documents Récupérés
Une autre manière utile d'évaluer les réponses sans avoir besoin de réponses de référence consiste à vérifier si la réponse est justifiée par (ou « ancrée dans ») les documents récupérés.

In [15]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[
        bool, ..., "Provide the score on if the answer hallucinates from the documents"
    ]

# Grade prompt
grounded_instructions = """Vous êtes un enseignant qui corrige un quiz. On vous donnera des FAITS et une RÉPONSE de l'ÉLÈVE. Voici les critères de notation à suivre :

(1) Assurez-vous que la RÉPONSE de l'ÉLÈVE est ancrée (grounded) dans les FAITS fournis.
(2) Assurez-vous que la RÉPONSE de l'ÉLÈVE ne contient pas d'informations "hallucinées" en dehors du cadre des FAITS.

Fidélité :
Une valeur "True" (Vrai) signifie que la réponse de l'élève répond à tous les critères.
Une valeur "False" (Faux) signifie que la réponse de l'élève ne répond pas à tous les critères.

Expliquez votre raisonnement étape par étape pour garantir que votre analyse et votre conclusion sont correctes. Évitez de donner simplement la réponse correcte dès le début."""

# Grader LLM
grounded_llm = ChatOllama(model=LLM_MODEL_NAME, temperature=0).with_structured_output(
    GroundedGrade, method="json_schema", strict=True
)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([
        {"role": "system", "content": grounded_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["grounded"]

### Retrieval relevance: Retrieved docs vs input

In [16]:
# Schéma de sortie pour la notation
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Expliquez votre raisonnement pour le score obtenu"]
    relevant: Annotated[
        bool, ..., "Vrai si les documents récupérés sont pertinents par rapport à la question, Faux sinon",
    ]

# Instructions de notation (Prompt)
retrieval_relevance_instructions = """Vous êtes un enseignant qui corrige un quiz. On vous donnera une QUESTION et un ensemble de FAITS fournis par l'élève. Voici les critères de notation à suivre :

(1) Votre objectif est d'identifier les FAITS qui sont totalement sans rapport avec la QUESTION.
(2) Si les faits contiennent n'importe quel mot-clé ou sens sémantique lié à la question, considérez-les comme pertinents.
(3) Il est ACCEPTABLE que les faits contiennent CERTAINES informations sans rapport avec la question, tant que le critère (2) est respecté.

Pertinence :
Une valeur "True" (Vrai) signifie que les FAITS contiennent des mots-clés ou une signification sémantique liés à la QUESTION et sont donc pertinents.
Une valeur "False" (Faux) signifie que les FAITS sont totalement sans rapport avec la QUESTION.

Expliquez votre raisonnement étape par étape pour garantir que votre analyse et votre conclusion sont correctes. Évitez de donner simplement la réponse correcte dès le début."""

# Initialisation du LLM de notation
retrieval_relevance_llm = ChatOllama(model=LLM_MODEL_NAME, temperature=0).with_structured_output(
    RetrievalRelevanceGrade, method="json_schema", strict=True
)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """Un évaluateur pour la pertinence des documents récupérés"""
    # Extraction du contenu des documents récupérés
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    # Préparation du prompt pour le juge
    answer = f"FAITS: {doc_string}\nQUESTION: {inputs['question']}"
    
    # Exécution de l'évaluateur
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

### Run evaluation

In [17]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

eval_data = pd.read_csv('data/tests/samples.csv', sep=";")

client = Client()
dataset_name = "Mon Dataset d'Evaluation"
if not client.has_dataset(dataset_name=dataset_name):
    client.create_dataset(dataset_name=dataset_name)

client.create_examples(
    inputs=[{"question": q} for q in eval_data["question"]],
    outputs=[{"answer": a} for a in eval_data["answer"]],
    dataset_name=dataset_name
)

experiment_results = client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="test",
    metadata={"version": "LCEL context, gpt-4-0125-preview"},
)

# experiment_results.to_pandas()

View the evaluation results for experiment: 'test-70025c8d' at:
https://smith.langchain.com/o/9dba41ba-edaf-4ddb-b5ed-eb28b05c0292/datasets/09f0b592-6a7e-488f-a980-a6b9d3507056/compare?selectedSessions=6c76d54b-c2f5-4c06-b067-916ed64158ac




50it [38:54, 46.69s/it]
